# Using Lazy Predict to test all models accuracy

In [3]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from lazypredict.Supervised import LazyClassifier
import pandas as pd
import random

seed = random.randint(1000, 9999)
print(seed)
# Load your dataset
df = pd.read_csv("ACME-HappinessSurvey2020.csv")

# Define the features (X) and the target variable (y)
X = df[['X1', 'X2', 'X3', 'X4', 'X5', 'X6']]
y = df['Y']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.2, random_state=seed)

# Initialize the LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

# Fit the models and make predictions
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# Print the models and their performance
print(models)


3539


100%|██████████| 31/31 [00:00<00:00, 42.35it/s]

[LightGBM] [Info] Number of positive: 51, number of negative: 49
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000456 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 31
[LightGBM] [Info] Number of data points in the train set: 100, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.510000 -> initscore=0.040005
[LightGBM] [Info] Start training from score 0.040005
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Ligh

# Running loop to get best 3 models for prediction

In [4]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from lazypredict.Supervised import LazyClassifier
import pandas as pd
import random

# Load your dataset
df = pd.read_csv("ACME-HappinessSurvey2020.csv")

# Define the features (X) and the target variable (y)
X = df[['X1', 'X2', 'X3', 'X4', 'X5', 'X6']]
y = df['Y']

# Initialize the best accuracy and models
best_accuracy = 0
best_models = None

# Loop until we reach the highest accuracy
while True:
    # Generate a random seed
    seed = random.randint(1000, 9999)
    print(f"Seed: {seed}")

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=.2, random_state=seed)

    # Initialize the LazyClassifier
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

    # Fit the models and make predictions
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)

    # Get the top 3 models
    top_models = models.head(3)

    # Get the average accuracy of the top 3 models
    avg_accuracy = top_models['Accuracy'].mean()

    # Check if the average accuracy is higher than the best accuracy
    if avg_accuracy > best_accuracy:
        best_accuracy = avg_accuracy
        best_models = top_models

    # Print the best models and their performance
    print(f"Best Models:")
    print(best_models)

    # Check if we have reached the highest accuracy
    if best_accuracy >= 0.95:
        break


Seed: 5795


100%|██████████| 31/31 [00:00<00:00, 72.02it/s]


[LightGBM] [Info] Number of positive: 52, number of negative: 48
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000029 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31
[LightGBM] [Info] Number of data points in the train set: 100, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.520000 -> initscore=0.080043
[LightGBM] [Info] Start training from score 0.080043
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

100%|██████████| 31/31 [00:00<00:00, 82.95it/s]


[LightGBM] [Info] Number of positive: 58, number of negative: 42
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000018 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31
[LightGBM] [Info] Number of data points in the train set: 100, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.580000 -> initscore=0.322773
[LightGBM] [Info] Start training from score 0.322773
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

...

[LightGBM] [Info] Number of positive: 60, number of negative: 40
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000018 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31
[LightGBM] [Info] Number of data points in the train set: 100, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.600000 -> initscore=0.405465
[LightGBM] [Info] Start training from score 0.405465
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

100%|██████████| 31/31 [00:00<00:00, 80.95it/s]


[LightGBM] [Info] Number of positive: 54, number of negative: 46
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000068 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31
[LightGBM] [Info] Number of data points in the train set: 100, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.540000 -> initscore=0.160343
[LightGBM] [Info] Start training from score 0.160343
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

  0%|          | 0/31 [00:00<?, ?it/s]


KeyboardInterrupt: 

# Observation

based on lazy predict analysis top 3 models are:

                        Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                    
LabelPropagation            0.88               0.89     0.89      0.89   
RandomForestClassifier      0.85               0.84     0.84      0.85   
BaggingClassifier           0.85               0.84     0.84      0.85   

# Label Propagation Test

In [ ]:
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import classification_report

seed = random.randint(1000, 9999)

lp = LabelPropagation()
lp.fit(X_train, y_train)
y_pred_lp = lp.predict(X_test)
print("LabelPropagation Classification Report:")
print(classification_report(y_test, y_pred_lp))


LabelPropagation Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.45      0.48        11
           1       0.62      0.67      0.65        15

    accuracy                           0.58        26
   macro avg       0.56      0.56      0.56        26
weighted avg       0.57      0.58      0.57        26



# Random Forest Classifier 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import random

# Set the random seed that gave the highest accuracy
seed = random.randint(1000, 9999)
rf = RandomForestClassifier(random_state=seed)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("RandomForestClassifier Classification Report:")
print(classification_report(y_test, y_pred_rf))


RandomForestClassifier Classification Report:
              precision    recall  f1-score   support

           0       0.45      0.45      0.45        11
           1       0.60      0.60      0.60        15

    accuracy                           0.54        26
   macro avg       0.53      0.53      0.53        26
weighted avg       0.54      0.54      0.54        26



# BaggingClassifier

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report
import random

# Set the random seed that gave the highest accuracy
seed = random.randint(1000, 9999)
bg = BaggingClassifier(random_state=seed)
bg.fit(X_train, y_train)
y_pred_bg = bg.predict(X_test)
print("BaggingClassifier Classification Report:")
print(classification_report(y_test, y_pred_bg))


BaggingClassifier Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.64      0.64        11
           1       0.73      0.73      0.73        15

    accuracy                           0.69        26
   macro avg       0.68      0.68      0.68        26
weighted avg       0.69      0.69      0.69        26



### Best of the three models is BaggingClassifier with highest accuracy of 0.64

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import random

# Set a random seed
seed = random.randint(1000, 9999)
print("Random seed:", seed)

# Load your dataset
df = pd.read_csv("ACME-HappinessSurvey2020.csv")


X = df[['X1', 'X2', 'X3', 'X4', 'X5', 'X6']]
y = df['Y']


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=seed)
print("X,y: ", X.shape, y)
# Label Propagation
lp = LabelPropagation()
lp.fit(X_train, y_train)
y_pred_lp = lp.predict(X_test)
cm_lp = confusion_matrix(y_test, y_pred_lp)
print("Label Propagation Confusion Matrix:")
print(cm_lp)

# Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=seed)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
cm_rf = confusion_matrix(y_test, y_pred_rf)
print("\nRandom Forest Classifier Confusion Matrix:")
print(cm_rf)

# Bagging Classifier
bg = BaggingClassifier(n_estimators=100, random_state=seed)
bg.fit(X_train, y_train)
y_pred_bg = bg.predict(X_test)
cm_bg = confusion_matrix(y_test, y_pred_bg)
print("\nBagging Classifier Confusion Matrix:")
print(cm_bg)


Random seed: 2694
X,y:  (126, 6) 0      0
1      0
2      1
3      0
4      0
      ..
121    1
122    1
123    1
124    0
125    0
Name: Y, Length: 126, dtype: int64
Label Propagation Confusion Matrix:
[[4 8]
 [5 9]]

Random Forest Classifier Confusion Matrix:
[[7 5]
 [5 9]]

Bagging Classifier Confusion Matrix:
[[5 7]
 [5 9]]


### Based on above results we can see that the best algorithm at predicting class 0 unhappy customers is Label propagation, since false negative is same for each algorithm